In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import glob
import cv2

import torch
from torch.nn import functional as F
from gfpgan.archs.arcface_arch import ResNetArcFace

from copy import deepcopy
def load_network(net, load_path, strict=True, param_key='params'):
        load_net = torch.load(load_path, map_location=lambda storage, loc: storage)
        # remove unnecessary 'module.'
        for k, v in deepcopy(load_net).items():
            if k.startswith('module.'):
                load_net[k[7:]] = v
                load_net.pop(k)
        net.load_state_dict(load_net, strict=strict)

arcface_resnet18 = ResNetArcFace(block='IRBlock', layers=[2, 2, 2, 2], use_se=False)
load_network(arcface_resnet18, '/home/viu/Work/GFPGAN/experiments/pretrained_models/arcface_resnet18.pth')
arcface_resnet18.cuda()
arcface_resnet18.eval()

ResNetArcFace(
  (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (prelu): PReLU(num_parameters=1)
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): IRBlock(
      (bn0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (prelu): PReLU(num_parameters=1)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): IRBlock(
      (bn0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(

In [2]:
tub_images = glob.glob('/data/datasets/Turbulence/Tubfaces95/300M/comparisons/EiGEN_145K/*.png')
tub_images.sort()
tub_images = tub_images[::3]

In [3]:
len(tub_images)

89

In [4]:
def gray_resize_for_identity(out, size=128):
    out_gray = (0.2989 * out[:, 0, :, :] + 0.5870 * out[:, 1, :, :] + 0.1140 * out[:, 2, :, :])
    out_gray = out_gray.unsqueeze(1)
    out_gray = F.interpolate(out_gray, (size, size), mode='bilinear', align_corners=False)
    return out_gray

In [5]:
from basicsr.utils import img2tensor, tensor2img
from torchvision.transforms.functional import normalize
from tqdm import tqdm

degs = []
reference_img_ths = []
tub_face_img_ths = []
for idx in tqdm(range(len(tub_images))):
    # reference_img = cv2.imread('/data/datasets/Turbulence/Tubfaces95/references/%02d.png'% (idx//3 + 1), cv2.IMREAD_UNCHANGED)
    reference_img = cv2.imread('/data/datasets/Turbulence/Tubfaces95/references/%02d.png'% (idx + 1), cv2.IMREAD_UNCHANGED)
    tub_img = cv2.imread(tub_images[idx], cv2.IMREAD_UNCHANGED)

    reference_img_th = img2tensor(reference_img / 255., bgr2rgb=True, float32=True)
    reference_img_th = reference_img_th.unsqueeze(0).cuda()
    reference_img_th = normalize(reference_img_th, (0.5, 0.5, 0.5), (0.5, 0.5, 0.5), inplace=True)
    reference_img_th = gray_resize_for_identity(reference_img_th)

    tub_face_img_th = img2tensor(tub_img / 255., bgr2rgb=True, float32=True)
    tub_face_img_th = tub_face_img_th.unsqueeze(0).cuda()
    tub_face_img_th = normalize(tub_face_img_th, (0.5, 0.5, 0.5), (0.5, 0.5, 0.5), inplace=True)
    tub_face_img_th = gray_resize_for_identity(tub_face_img_th)

    with torch.no_grad():
        tub_face_img_th = arcface_resnet18(tub_face_img_th)
        reference_img_th = arcface_resnet18(reference_img_th)
    reference_img_ths.append(reference_img_th)
    tub_face_img_ths.append(tub_face_img_th)
    deg = F.cosine_similarity(reference_img_th, tub_face_img_th).item()
    degs.append(deg)

100%|██████████| 89/89 [00:02<00:00, 34.13it/s]


In [6]:
sum(degs) / len(degs)

0.35169839524151236

In [7]:
tub_face_img_ths = torch.cat(tub_face_img_ths)
reference_img_ths = torch.cat(reference_img_ths)

In [8]:
similarity = torch.mm(reference_img_ths, tub_face_img_ths.transpose(0, 1))
top_inds = torch.argsort(-similarity)

In [9]:
similarity.shape

torch.Size([89, 89])

In [10]:
# top1
correct_num = 0
for i in range(tub_face_img_ths.size(0)):
    j = top_inds[i, 0]
    # if j//3 == i//3:
    if j == i:
        correct_num += 1
print("top1 = {}".format(correct_num / tub_face_img_ths.size(0)))

top1 = 0.6179775280898876


In [11]:
# top3
correct_num = 0
for i in range(tub_face_img_ths.size(0)):
    # j = top_inds[i, 0:9]
    # if i//3 in j//3:
    j = top_inds[i, 0:3]
    if i in j:
        correct_num += 1
print("top3 = {}".format(correct_num / tub_face_img_ths.size(0)))

top3 = 0.7415730337078652


In [12]:
# top5
correct_num = 0
for i in range(tub_face_img_ths.size(0)):
    j = top_inds[i, 0:5]
    if i in j:
    # j = top_inds[i, 0:15]
    # if i//3 in j//3:
        correct_num += 1
print("top5 = {}".format(correct_num / tub_face_img_ths.size(0)))

top5 = 0.7865168539325843
